In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
df1 = pd.read_csv('data/query_output.csv', usecols=['gvkey', 'permno', 'public_date', 'bm', 'pcf'])
df1.loc[:, 'cfp'] = 1 / df1['pcf']
df2 = pd.read_csv('data/prices.csv') 
df2.rename(columns={'PERMNO': 'permno', 'date': 'public_date'}, inplace=True)
df = pd.merge(df1, df2, on=['permno', 'public_date'], how='outer')
del df1, df2
df.index = pd.to_datetime(df['public_date'])
df = df.loc[:, ['permno', 'TICKER', 'COMNAM', 'public_date', 'bm', 'cfp', 'PRC', 'VOL', 'RET', 'SHROUT']].dropna()
df.loc[:, 'mcap'] = df['PRC'] * df['SHROUT']

def make_quantiles(df, column):
    df = df.sort_values(column, ascending=True)
    df['quantile'] = pd.qcut(df[column], 5, labels=False)
    return df

df = df.groupby(df.index).apply(make_quantiles, 'cfp').dropna().reset_index(drop=True)
df.loc[:, 'portfolio'] = 0.0
df['RET'] = pd.to_numeric(df['RET'], errors='coerce')
df['mcap'] = pd.to_numeric(df['mcap'], errors='coerce')

def calculate_portfolio_returns(df):
    df.loc[:, 'portfolio'] = df['RET'] * df['mcap'] / df['mcap'].sum()
    df.loc[:, 'portfolio'] = df['portfolio'].sum().item()
    return df


df = df.groupby(['public_date', 'quantile']).apply(calculate_portfolio_returns).reset_index(drop=True)

df


/var/folders/r0/zk7h1dpx693gc15c5y61js8w0000gn/T/ipykernel_49049/1612498734.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['public_date', 'quantile']).apply(calculate_portfolio_returns).reset_index(drop=True)


,permno,TICKER,COMNAM,public_date,bm,cfp,PRC,VOL,RET,SHROUT,mcap,quantile,portfolio
0,27596,SB,SALOMON INC,1990-01-31,0.931,-16.129032,21.250,83049.0,-0.090909,124929.0,2.654741e+06,0,-0.083499
1,70519,PA,PRIMERICA CORP NEW,1990-01-31,0.791,-2.673797,26.125,79273.0,-0.080526,95999.0,2.507974e+06,0,-0.083499
2,68304,BSC,BEAR STEARNS COMPANIES INC,1990-01-31,0.671,-2.421308,13.000,40221.0,-0.028037,91394.0,1.188122e+06,0,-0.083499
3,48071,JPM,MORGAN J P & CO INC,1990-01-31,0.477,-1.769912,36.000,137502.0,-0.181818,183763.0,6.615468e+06,0,-0.083499
4,48223,MHC,MANUFACTURERS HANOVER CORP,1990-01-31,0.952,-1.557632,29.875,86839.0,-0.098113,65964.0,1.970674e+06,0,-0.083499
...,...,...,...,...,...,...,...,...,...,...,...,...,...
189081,27633,R,RYDER SYSTEMS INC,2021-12-31,0.840,0.490196,82.430,115077.0,-0.007824,53693.0,4.425914e+06,4,0.053595
189082,70519,C,CITIGROUP INC,2021-12-31,1.313,0.507614,60.390,5170579.0,-0.051962,1984267.0,1.198299e+08,4,0.053595
189083,60599,LUMN,LUMEN TECHOLOGIES INC,2021-12-31,1.168,0.511771,12.550,2379180.0,0.017018,1023894.0,1.284987e+07,4,0.053595
189084,90805,DISCA,DISCOVERY INC,2021-12-31,0.993,0.619963,23.540,1364590.0,0.011603,169000.0,3.978260e+06,4,0.053595


In [3]:
portfolios = df.groupby(['public_date', 'quantile'])['portfolio'].mean().unstack()
portfolio_values = (1 + portfolios).cumprod()
portfolio_values

quantile,0,1,2,3,4
public_date,,,,,
1990-01-31,0.916501,0.927831,0.953581,0.928519,0.940381
1990-02-28,0.944145,0.932935,0.964207,0.950673,0.963100
1990-04-30,0.936209,0.928719,0.951089,0.919318,0.919306
1990-05-31,1.068946,1.045283,1.038196,0.987217,0.996858
1990-07-31,1.050948,1.045437,1.041686,1.002879,0.965535
...,...,...,...,...,...
2021-06-30,1057.105015,134.310184,32.966050,11.301987,3.925315
2021-08-31,1113.010866,138.679710,33.809039,11.548433,3.990102
2021-09-30,1087.196630,131.274533,32.038899,11.143361,3.913837


In [4]:
fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=('Portfolio Returns', 'Portfolio Values'),
                    vertical_spacing=0.1)
for i in range(5):
    fig.add_trace(
        go.Scatter(x=portfolios.index, y=portfolios[i], mode='lines', name=f'Returns Q{i+1}'),
        row=1, col=1
    )
for i in range(5):
    fig.add_trace(
        go.Scatter(x=portfolio_values.index, y=portfolio_values[i], mode='lines', name=f'Values Q{i+1}'),
        row=2, col=1
    )
fig.update_layout(
    height=1000,  # Increased height to accommodate two subplots
    title_text='Portfolio Performance Analysis'
)
fig.update_xaxes(title_text='Date', row=1, col=1)
fig.update_yaxes(title_text='Return', row=1, col=1)
fig.update_xaxes(title_text='Date', row=2, col=1)
fig.update_yaxes(title_text='Value', row=2, col=1)

fig.show()